In [0]:
# get_data = "https://insideairbnb.com/get-the-data/"


In [0]:
# dbutils.fs.ls(path_raw)

[FileInfo(path='dbfs:/FileStore/Projeto_Analytics/data/raw/calendar_csv.gz', name='calendar_csv.gz', size=33229501, modificationTime=1730681160000),
 FileInfo(path='dbfs:/FileStore/Projeto_Analytics/data/raw/listings_csv.gz', name='listings_csv.gz', size=18200635, modificationTime=1730681165000),
 FileInfo(path='dbfs:/FileStore/Projeto_Analytics/data/raw/reviews_csv.gz', name='reviews_csv.gz', size=86504369, modificationTime=1730681172000)]

In [0]:
# dbutils.fs.ls(path_raw)[0].name

'calendar_csv.gz'

In [0]:
def to_raw(get_raw, path_to_raw, file_name):

    new_data = (spark.read.
                option("multiline", "true").
                option("quote", '"').
                option("escape", '"').
                csv(get_raw, header=True))
    
    for i in dbutils.ls(path_to_raw):
        if len(i) > 0:
            if file_name in i.name:
                current_data = (spark.read.
                                option("multiline", "true").
                                option("quote", '"').
                                option("escape", '"').
                                csv(path_to_raw, header=True))
                
                current_data = current_data.unionByName(new_data)

                current_data = current_data.dropDuplicates()

        else:
            current_data = new_data.dropDuplicates()

    (current_data.write.mode("overwrite")
                       .option("header", "true")
                       .option("compression", "gzip")
                       .csv(path_to_raw))